## 准备工作

智谱API申请，大家可以请前往 https://open.bigmodel.cn/dev/api#glm-3-turbo 申请智谱API

安装 Milvus 向量库

### 基础环境

* python 3.9.6
* nodejs v18.17.1

## Milvus 向量数据库

Milvus 是一个高度灵活、可靠且速度极快的云原生开源向量数据库。它为 embedding 相似性搜索和 AI 应用程序提供支持，并努力使每个组织都可以访问向量数据库。 Milvus 可以存储、索引和管理由深度神经网络和其他机器学习（ML）模型生成的十亿级别以上的 embedding 向量。
它具备高可用、高性能、易拓展的特点，用于海量向量数据的实时召回。

### 为什么选择使用 Milvus
* 高性能：性能高超，可对海量数据集进行向量相似度检索。
* 高可用、高可靠：Milvus 支持在云上扩展，其容灾能力能够保证服务高可用。
* 混合查询：Milvus 支持在向量相似度检索过程中进行标量字段过滤，实现混合查询。
* 开发者友好：支持多语言、多工具的 Milvus 生态系统。

安装 Milvus 请参考 https://milvus.io/docs/install_standalone-docker.md 官方文档

当前使用版本 <span style="color: red">v2.2.11</span>

## LangChain 框架

LangChain 是一个开源框架，用于构建基于大型语言模型（LLM）的应用程序。LangChain 提供各种工具和抽象，以提高模型生成的信息的定制性、准确性和相关性。例如，开发人员可以使用 LangChain 组件来构建新的提示链或自定义现有模板。LangChain 还包括一些组件，可让 LLM 无需重新训练即可访问新的数据集。

当前项目使用了 LangChain 中的文档加载器、ChatZhipuAI 组件。LangChain 已经对接了智谱的大语言模型，可以直接使用。

当前使用版本 <span style="color: red">0.1.17</span>

#### LangChain 核心模块概览
目前围绕 LangChain 框架核心模块主要有六个

* 1. LLM 和提示：LangChain 使管理提示、优化它们以及为所有 LLM 创建通用界面变得容易。此外，它还包括一些用于处理 LLM 的便捷实用程序。
* 2. 链(Chain)：这些是对 LLM 或其他实用程序的调用序列。LangChain 为链提供标准接口，与各种工具集成，为流行应用提供端到端的链。
* 3. 数据增强生成：LangChain 使链能够与外部数据源交互以收集生成步骤的数据。例如，它可以帮助总结长文本或使用特定数据源回答问题。
* 4. Agents：Agents 让 LLM 做出有关行动的决定，采取这些行动，检查结果，并继续前进直到工作完成。LangChain 提供了代理的标准接口，多种代理可供选择，以及端到端的代理示例。
* 5. 内存：LangChain 有一个标准的内存接口，有助于维护链或代理调用之间的状态。它还提供了一系列内存实现和使用内存的链或代理的示例。
* 6. 评估：很难用传统指标评估生成模型。这就是为什么 LangChain 提供提示和链来帮助开发者自己使用 LLM 评估他们的模型。

文档加载器

<img src="./image/langchain_file.png" style="width:50%" /> 

智谱AI

<img src="./image/langchain_zhipu.png" style="width:50%" />

## 目标

通过将指定文档转换成向量，然后可以根据问题从向量库中查找最相似的文档，并根据文档内容通过AI回答提出的问题。

可以先看下当前准备的文档信息

## 创建 Milvus 向量库数据集和写入向量数据

首先需要在向量数据库中创建集合并添加索引

In [3]:
from pymilvus import Collection, FieldSchema, DataType, CollectionSchema, connections

connections.connect(alias='main', host="127.0.0.1", port=19530)

def create_index():
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), # 自增ID
        FieldSchema(name="document_ebm", dtype=DataType.FLOAT_VECTOR, dim=1024), # 转换后的向量字段，我们使用的是智谱的向量模型，它的维度是1024，也可以采用开源模型，每种模型的维度可能不一样，比如还有有768位度或者更高维度的向量模型
        FieldSchema(name="document_text", dtype=DataType.VARCHAR, max_length=65535), # 原始文档信息
    ]
    schema = CollectionSchema(fields=fields)
    # 创建集合
    collection = Collection(name='documents', schema=schema, using='main')
    # 添加索引
    index_params = {
        "index_type": "HNSW",
        "metric_type": "IP",
        "params": {
            "M": 32,
            "efConstruction": 512
        }
    }
    collection.create_index(
        field_name="document_ebm",
        index_params=index_params
    )

    collection.create_index(
        field_name="document_text",
        index_params={
            "index_type": "marisa-trie"
        }
    )
    # 将数据加载到内存中
    collection.load()

### 索引类型
* HNSW
* FLAT
* IVF_FLAT
* RNSG
等 

### HNSW 索引类型
HNSW 是一种基于图的索引算法。它按照一定的规则为图像构建多层导航结构。在这种结构中，上层更加稀疏，节点之间的距离更远；较低的层更密集，节点之间的距离更近。搜索从最上层开始，找到本层中距离目标最近的节点，然后进入下一层开始另一次搜索。经过多次迭代，可以快速逼近目标位置。

特点
* 非常高速的查询
* 召回率高、精准度高
* 磁盘、内存占用大于原始数据，适用于小数据量，内存充足的计算

### 向量相似度算法
* 内积 (IP)
* 欧氏距离 (L2)

### IP 用于衡量向量相似度的度量类型
两个嵌入之间的IP距离


## 将文档转换成向量

这里利用 langchain 框架的文档加载器来加载需要的文档资源

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

def loadDocumentation(path: str, glob: str = "*.text"):
    """读取目录下指定文件"""
    loader = DirectoryLoader(path=path, glob=glob)
    return loader.load()

调用智谱AI向量模型将文本转换成Embedding，并将转换后的Embedding写入到Milvus 向量数据库中。

In [2]:
from zhipuai import ZhipuAI
from pymilvus import Collection, connections
import os

ZHIPUAI_API_KEY = os.environ.get('ZHIPUAI_API_KEY') # 替换成您自己的 key
client = ZhipuAI(api_key=ZHIPUAI_API_KEY)

connections.connect(alias='main', host="127.0.0.1", port=19530)
collection = Collection(name='documents', using='main')


def documentEmbedding(docs):
    document_ebm = []
    document_text = []
    for doc in docs:
        embedding = getEmbedding(doc.page_content)
        document_ebm.append(embedding)
        document_text.append(doc.page_content)

    documents = [document_ebm, document_text]
    # 批量写入数据
    res = collection.insert(data=documents)
    collection.flush()
    # 打印结果
    print(f"插入行数 -> {res.insert_count}")


def getEmbedding(text):
    """ 调用智谱的 embedding-2 模型，将文本转换成 embedding 结果 """
    response = client.embeddings.create(
        model="embedding-2",
        input=text,
    )
    return response.data[0].embedding


if __name__ == '__main__':
    # 读取文档
    docs = loadDocumentation("./text/", "*.text")
    documentEmbedding(docs)

#### 向量转换后的结果
![image.png](./image/emb.png)

## 根据提出的问题在向量库中查找最相似的结果
Milvus 中的向量相似度搜索会计算查询向量与具有指定相似度度量的集合中的向量之间的距离，并返回最相似的结果。通过指定过滤标量字段或主键字段的布尔表达式，您可以执行混合搜索。

首先将问题调用智谱的 embedding-2 模型，将问题转成 embedding，再利用问题的 embedding 去向量库中查询最相似的结果，返回 topk 1 个结果

In [ ]:
def getDocuments(question: str):
    query_vector = getEmbedding(question)
    search_params = {"metric_type": "IP", "params": {"nprobe": 16}, "offset": 0}
    results = collection.search(data=[query_vector], anns_field='document_ebm', param=search_params, limit=1, output_fields=['document_text'])
    documents = []
    for result in results:
        for r in result:
            document_text = str(r.entity.get('document_text'))
            documents.append(document_text)

    return documents

匹配结果

|问题<div style="width:200px">|文档|分数|
| --- | --- | --- |
|为什么SQL语句不能写1=1？|'最近看几个老项目的SQL条件中使用了1=1，想想自己也曾经这样写过，略有感触，特别拿出来说道说道。\n\n编写SQL语句就像炒菜，每一种调料的使用都可能会影响菜品的最终味道，每一个SQL条件的加入也可能会影响查询的执行效率。那么 1=1 存在什么样的问题呢？为什么又会使用呢？\n\n为什么会使用 1=1？\n\n在动态构建SQL查询时，查询条件往往都是动态的，最终执行时可能会使用不同的条件。这时候，他们就会使用“1=1”作为一个始终为真的条件，让接下来的所有条件都可以方便地用“AND”连接起来，就像是搭积木的时候先放一个基座，其他的积木块就都可以在这个基座上叠加。\n\n1=1 带来的问题\n\n性能问题？\n\n我们先来了解一下数据库查询优化器的工作原理。查询优化器就像是一个聪明的图书管理员，它知道如何最快地找到你需要的书籍。当你告诉它所需书籍的特征时，它会根据这些信息选择最快的检索路径。比如你要查询作者是“谭浩强”的书籍，它就选择先通过作者索引找到书籍索引，再通过书籍索引找到对应的书籍，而不是费力的把所有的书籍遍历一遍。\n\n但是，如果我们告诉它一些无关紧要的信息，比如“我要一本书，它是一本书”，这并不会帮助管理员更快地找到书，反而可能会让他觉得困惑。一个带有“1=1”的查询可能会让数据库去检查每一条记录是否满足这个始终为真的条件，这就像是图书管理员不得不检查每一本书来确认它们都是书一样，显然是一种浪费。\n\n你可能会说：数据库没有这么傻吧？\n\n确实，这实际上可能不会产生问题，因为现代数据库的查询优化器已经非常智能，它们通常能够识别出像 1=1 这样的恒真条件，并在执行查询计划时优化掉它们。在许多情况下，即使查询中包含了1=1，数据库的性能也不会受到太大影响，优化器会在实际执行查询时将其忽略。\n\n但是优化器并不是万能的。在某些复杂的查询场景中，即使是简单的 1=1 也可能对优化器的决策造成不必要的影响，比如导致全表扫描。\n\n代码质量\n\n另外从代码质量的角度，我们也需要避免在查询中包含 1=1，有以下几点考虑：\n\n代码清晰性：即使数据库可以优化掉这样的条件，但对于阅读SQL代码的人来说，1=1可能会造成困惑。代码的可读性和清晰性非常重要，特别是在团队协作的环境中。\n\n习惯养成：即使在当前的数据库系统中1=1不会带来性能问题，习惯了写不必要的代码可能会在其他情况下引入实际的性能问题。比如，更复杂的无用条件可能不会那么容易被优化掉。\n\n跨数据库兼容性：不同的数据库管理系统（DBMS）可能有不同的优化器能力。一个系统可能轻松优化掉1=1，而另一个系统则可能不那么高效。编写不依赖于特定优化器行为的SQL语句是一个好习惯。\n\n编写尽可能高效、清晰和准确的SQL语句，不仅有助于保持代码的质量，也让代码具有更好的可维护性和可扩展性。'|0.6723401546478271|
|为什么SQL语句不能写1=1？|'语言模型生成词元的过程是逐个进行的。可以把语言模型（具体来说，是仅解码器文本Transformer模型，但本文其余部分将其简称为LLM）理解为一个函数，它以词元作为输入并生成一个概率数组，用于表示词汇表中所有词元的概率（通常词汇表中有50-250K词元，每个词元由几个字母组成）。然后，程序根据这些概率从所有词元中进行采样，以指导采样过程，并生成下一个词元，这一过程会重复进行。这意味着，生成文本序列时不可能存在并行性——生成过程可以逐个词元地进行建模。\n\n总的来说，语言模型在处理词元时会进行两种类型的操作：矩阵-向量乘法，其中一个大矩阵（例如8192x8192）与一个向量相乘，得到另一个向量，以及注意力计算。在生成过程中，模型不仅可以看到当前词元的状态，还可以看到序列中所有先前词元的内部状态，其中包括用户在提示中编写的词元以及模型生成的词元。这些状态存储在一个称为“KV-cache（键值缓存）”的结构中，它本质上是文本中每个先前位置的一组Key和Value向量。注意力机制会获取当前词元生成的query向量，计算它与所有先前位置的所有K向量的点积，然后对所得的一组标量进行归一化，并通过对所有先前位置的所有V向量进行加权求和得出一个V向量，使用点积作为注意力分数。\n\n现在，矩阵-向量乘法和注意力计算都有一个重要的特点：对于从矩阵或KV缓存中读取的每个元素，我们需要执行少量的浮点运算。矩阵-向量乘法对每个矩阵元素执行一次乘加运算（2 FLOPs）；而注意力计算对每个Key元素进行一次乘加计算来计算点积，并对每个V元素进行一次乘加计算来计算加权和。\n\n现代CPU和GPU进行ALU运算的速度（乘法、加法）远高于它们从内存读取输入的速度。例如：\n\nAMD Ryzen 7950X的内存带宽为67 GB/s，浮点运算能力为2735 GFLOPS，FLOP:字节读取比为40:1。\n\nNVidia GeForce RTX 4090的内存带宽为1008 GB/s，运算能力为83 TFLOPS，FLOP:字节读取比为82:1。\n\nNVidia H100 SXM（一款数据中心显卡）的内存带宽为3350 GB/s，运算能力为67 TFLOPS，FLOP:字节读取比看似为20:1；然而，对于类似矩阵乘法的问题，张量核心提供了约494 TFLOPS的运算能力，因此FLOP:字节读取比为147:1，不考虑稀疏性。\n\n对于较小的浮点数，如FP16或FP8，情况则变得更糟：H100张量核心在处理密集FP8矩阵时，理论吞吐量为1979 TFLOPS，这使得FLOP:字节读取比达到590:1。毋庸置疑，无论采用哪种配置，无论是否使用张量核心或使用何种浮点格式，ALU单元的资源都是充裕的。\n\n因此，任何只需要对每个元素执行两次操作的问题必定会受限于带宽，我们应该能够通过模型配置、KV缓存大小以及可用带宽来估算运行推理过程所需的最短时间。'|0.19971448183059692|
|为什么SQL语句不能写1=1？|'大模型第一次达到了「类人」水平，用户可以用自然语言与计算机进行交流了。那么为了达到事半功倍的效果，我们就要精进提问的技巧，这便是Prompt Engineering。如何做呢？要把大模型视为精通某一领域的专业顾问，要以与专业顾问（比如导游，咨询师）对话的角度来看待问题。\n\n比如说马上五一假期了，想要去上海玩，你问大模型『五一假期去上海玩，推荐一些行程』，大模型肯定会给你一个中规中矩的万能日程。这不能怪大模型，如果你如此问一个导游，负责的导游会直接反问你一大堆细节问题，因为这是行程所必须的；或者也是随便推荐一些大家都知道的地方。但肯定 是没有参考价值的。\n\n为了达到最好的「沟通」效果，避免「Garbage In, Garbage Out」，就需要：\n\n视为在与专业人士对话，也就是要拟定大模型的角色\n\n一次对话尽可能的专注于一个问题或者一个主题\n\n把问题描述清楚，需要的关键要素都列清楚，比如时间，地点人物，关键事件等等\n\n限定问题，也就是说要尽可能多的补充问题细节，限定问题需要的回答，比如说可以做什么，不可以做什么，需要什么是时间点\n\n恰当的分隔，要多多使用标点符号对提问进行分隔，这样更有利于大模型抓住重点\n\n可以发现，这其实是沟通的技巧，抛开大模型，我们与正常的人沟通时，不也应该这样做吗？平时里的闲聊除外，正式的与人沟通时也应该使用这些技巧以达到最好的沟通效果。所以，最先需要掌握就是沟通技巧中的提问技巧。\n\n继续我们上面的问题，使用上面的方法来优化提示：『五一假期，从南京出发，坐高铁，目的地上海，亲子3日游，不去迪士尼，不去动物园，安排详细行程』。这回得到的回答肯定有更大的参考价值。\n\n大语言模型较以往的AI最大的进步在于超长的上下文记忆能力，这是它能达到「类人」水平的最主要的原因。那么在与大模型进行对话的时候，就要及时的补充上下文信息。一方面，你不可能一次性的把提问信息全都写全写对，那么一旦想到新的提示，就要及时的给到大模型；另外一方面，大模型有时候会胡编乱造，甚至胡言乱语（大模型都有一个叫做温度temperature的参数以控制这方面的行为），这并不是bug，而是语言创造力的一个体现。那么，一旦发现大模型跑偏了，就需要及时提供新的提示，补充上下文信息，对大模型进行纠正。\n\n另外，就是如果感觉问题不太好理解，或者对输出有特殊的要求，还有一个补充上下文的办法就是给一个输出的示例，大模型是能够很好的捕获这一点的，并且这个对让大模型输出优质的回答非常有帮助。\n\n不用担心溢出哈（就是输入字数太多，导致大模型理解不了），都4202年了，现在的大模型的上下文能力至少在4096个Token以上，对于大部分的常规问题来说足够了。'|0.19669339060783386|
|为什么SQL语句不能写1=1？|'JSP内置对象是一组自动可用的对象，这些对象代表了与请求、响应和JSP页面容器环境相关的各种对象。你不需要显式地创建这些对象，它们由JSP容器提供。这些内置对象提供了对HTTP请求-响应模型的直接支持，并且简化了一些常见的任务比如获取用户数据，跟踪会话等。\n\nJSP 内置对象列表\n\nrequest: HttpServletRequest 对象，表示客户端发起的请求。\n\nresponse: HttpServletResponse 对象，表示对客户端的响应。\n\nout: JspWriter 对象，用于向客户端发送输出。\n\nsession: HttpSession 对象，用于管理与特定用户的会话。\n\napplication: ServletContext 对象，代表整个Web应用程序的上下文。\n\nconfig: ServletConfig 对象，包含了Servlet的初始化参数。\n\npageContext: PageContext 对象，提供了对JSP页面所有对象和命名空间的访问。\n\npage: 表示当前的Servlet实例，通常对用户不是太有用。\n\nexception: 表示JSP页面中发生的异常对象。\n\n域（Scopes）\n\n在JSP中，域指的是数据可以存活的时间和范围，也就是说，在域中存放的对象可以被Web应用程序中的其他组件访问。JSP 支持以下四种域：\n\nPage Scope: 仅限于本页内可用。\n\nRequest Scope: 与一个请求的生命周期一致。\n\nSession Scope: 与用户的一个会话期间一致。\n\nApplication Scope: 对所有用户的会话期间一致，只要应用程序启动，数据就会存在。\n\n作用域从小到大的排序是：\n\nPage Scope：范围最小，只在当前页面有效，当页面处理完毕后，page scope 中的属性就不再可用了。\n\nRequest Scope：范围较小，只在一次请求中有效，请求被转发到其他资源（比如另一个JSP，Servlet等）时，这些属性仍然可用。\n\nSession Scope：范围较大，跨越多次请求，在一个用户的多次请求之间共享数据，通常用于维护用户会话期间的状态。\n\nApplication Scope：范围最大，跨越整个Web应用程序，所有的用户共享在此域中的属性，只要应用程序启动，这些属性就一直存在。'|0.11639922857284546|
|为什么SQL语句不能写1=1？|'在现代Web开发中，一个Node.js应用的构建往往离不开各种依赖，比如库、辅助工具或其他工具包。以一个典型的React项目为例，当你想为项目添加路由功能时，你需要安装如react-router-dom这样的包。类似这样的需求在开发过程中屡见不鲜，而这就是为什么我们需要一个包管理器来帮助我们管理这些依赖。\n\n默认情况下，Node.js安装时会自带NPM（Node Package Manager），作为最初的包管理工具，它为我们的开发提供了极大的便利。然而，随着项目的不断演进和需求的日益增长，仅仅依赖NPM可能无法完全满足我们的所有需求。\n\n因此，了解不同的包管理器，以及它们各自的优势和局限，对于选择最适合自己项目的工具至关重要。下面，我们将探讨包管理器的几个关键作用，帮助你更好地理解它们的价值。\n\n依赖管理\n\n包管理器的核心功能之一是依赖管理。它负责安装、更新和管理项目所需的所有外部依赖，确保依赖版本的正确性和在项目中的可用性。这不仅节省了开发者大量的时间，还避免了因手动管理依赖而可能导致的错误。\n\n安装便捷\n\n从下载命令的提供到本地机器上的依赖、漏洞与安全性评估，这一系列复杂的管理工作都由包管理器自动完成。这大大简化了项目的初始化和后续的依赖更新过程。\n\n脚本与命令\n\n通过在package.json文件中定义额外的脚本命令，包管理器使得常见的开发流程（如启动服务器、运行测试、构建资源等）变得简单快捷。这些命令可以通过包管理器的命令行工具（CLI）直接执行，极大地提高了开发效率。\n\n安全保障\n\n包管理器还提供了工具来扫描已知的安全漏洞，例如NPM的npm audit命令。它们还关注依赖锁定、包签名和验证等安全性和安全措施，从而保护你的项目免受潜在的安全威胁。\n\n通过了解不同包管理器的这些核心功能，你将更加有信心地选择适合自己项目需求的工具。不管是NPM、Yarn还是PNPM，它们都旨在使你的开发工作流程更加顺畅，帮助你更高效、更安全地管理项目依赖。'|0.05798647552728653|

langchain 已经集成了智谱的API接口，只需要导入 ChatZhipuAI 

定义AI的 prompt

* System prompt """请根据以下内容回答提出的问题，不要使用自身知识回答问题，回答的越详细越好""" AI 会根据自身的知识库来回答提出的问题，在 System prompt 中拒绝AI自身的回答
* User prompt """请基于以下内容:
\"\"\"
{context}
\"\"\"
来回答用户提出的问题，请用中文回答。

问题: {question}"""

使用 HumanMessagePromptTemplate 模版对象，传入 question 和 context，生成 user 的 prompt

调用智谱的AI模型，得到流式输出结果

In [ ]:
from langchain.callbacks import AsyncIteratorCallbackHandler
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate
import asyncio
import json

async def sse_chat_bot(query: str):
    documents = getDocuments(query)
    print("查询到的文档")
    print(f"\033[92m{documents}\033[0m")
    callback = AsyncIteratorCallbackHandler()
    streaming_chat = ChatZhipuAI(
        model="glm-3-turbo",
        api_key=ZHIPUAI_API_KEY,
        temperature=0.01,
        streaming=True,
        callback_manager=CallbackManager([callback]),
    )
    # 定义问题模版
    general_user_template = general_user_template = """
请基于以下内容:
\"\"\"
{context}
\"\"\"
来回答用户提出的问题，请用中文回答。

问题: {question}"""

    human_message = HumanMessagePromptTemplate.from_template(general_user_template, input_variables=["question", "context"])
    messages = [
        SystemMessage(content="请根据以下内容回答提出的问题，不要使用自身知识回答问题，回答的越详细越好"),
        human_message.format(question=query, context='/n'.join(documents)),
    ]

    agenerate = streaming_chat.agenerate([messages])
    finalAnswer = asyncio.create_task(agenerate)
    async for token in callback.aiter():
        yield json.dumps({
            'output': token,
            'end': False
        }, ensure_ascii=False)

    result = await finalAnswer

    yield json.dumps({
        'output': result.generations[0][0].text,
        'end': True
    }, ensure_ascii=False)

## Demo 演示

可以前往 https://github.com/zxing258974/RagDemo 下载当前演示项目的源码

本项目使用了前后端分离，前端使用了 VUE3 ，位于 RagPage/ 目录下。后端使用了 FastApi 框架。更多的项目信息可以参考 README.md 文件